Clonar Darknet desde repos Github

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14691, done.
remote: Total 14691 (delta 0), reused 0 (delta 0), pack-reused 14691
Receiving objects: 100% (14691/14691), 13.27 MiB | 23.59 MiB/s, done.
Resolving deltas: 100% (9995/9995), done.


Se compila darknet para trabajar con GPU.



In [ ]:
%cd darknet
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!cat Makefile
!make

/content/darknet
GPU=1
CUDNN=0
CUDNN_HALF=0
OPENCV=1
AVX=0
OPENMP=0
LIBSO=0
ZED_CAMERA=0
ZED_CAMERA_v2_8=0

# set GPU=1 and CUDNN=1 to speedup on GPU
# set CUDNN_HALF=1 to further speedup 3 x times (Mixed-precision on Tensor Cores) GPU: Volta, Xavier, Turing and higher
# set AVX=1 and OPENMP=1 to speedup on CPU (if error occurs then set AVX=0)
# set ZED_CAMERA=1 to enable ZED SDK 3.0 and above
# set ZED_CAMERA_v2_8=1 to enable ZED SDK 2.X

USE_CPP=0
DEBUG=0

ARCH= -gencode arch=compute_35,code=sm_35 \
      -gencode arch=compute_50,code=[sm_50,compute_50] \
      -gencode arch=compute_52,code=[sm_52,compute_52] \
	    -gencode arch=compute_61,code=[sm_61,compute_61]

OS := $(shell uname)

# GeForce RTX 3070, 3080, 3090
# ARCH= -gencode arch=compute_86,code=[sm_86,compute_86]

# Kepler GeForce GTX 770, GTX 760, GT 740
# ARCH= -gencode arch=compute_30,code=sm_30

# Tesla A100 (GA100), DGX-A100, RTX 3080
# ARCH= -gencode arch=compute_80,code=[sm_80,compute_80]

# Tesla V100
# ARCH= -genco

se importa la dataset desde google drive para hacer uso de los archivos .names .data y los txt (conserva las rutas absolutas)

In [ ]:
import os.path
import shutil
from google.colab import drive

if not os.path.exists('/content/gdrive'):
  drive.mount('/content/gdrive')
  
DWD = '/content/darknet/data/'
DATASET = '/content/gdrive/My Drive/ml2/Dataset-yolo.rar' 

shutil.copy(DATASET, DWD)


'/content/darknet/data/Dataset-yolo.rar'

Se descomprime el dataset.zip (o .rar)

In [ ]:
!unrar x data/Dataset-yolo.rar -d data/



UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from data/Dataset-yolo.rar

Creating    data/Dataset-yolo                                         OK
Creating    data/Dataset-yolo/data                                    OK
Creating    data/Dataset-yolo/data/img                                OK
Extracting  data/Dataset-yolo/data/img/00000000.jpg                        0%  OK 
Extracting  data/Dataset-yolo/data/img/00000000.txt                        0%  OK 
Extracting  data/Dataset-yolo/data/img/00000001.jpg                        0%  OK 
Extracting  data/Dataset-yolo/data/img/00000001.txt                        0%  OK 
Extracting  data/Dataset-yolo/data/img/00000002.jpg                        0%  OK 
Extracting  data/Dataset-yolo/data/img/00000002.txt                        0%  OK 
Extracting  data/Dataset-yolo/data/img/00000003.jpg                        0%  OK 
Extracting  data/Dataset-yolo

Se configura lineas de codigo  del archivo yolov3.cfg para adecuarla al numero de clases entrenadas o a entrenar, en este caso es 1.

se configura los batch = 64

subdivision =32

max_batches = se calcula con numero de clases * 2000

steps = 80% de los max_batches, 90%

classes = numero de clases

filters = se (calcula con numerode clases + 5)*3



In [ ]:
!sed -i 's/batch=1/batch=64/g' cfg/yolov3.cfg
!sed -i 's/subdivisions=1/subdivisions=32/g' cfg/yolov3.cfg
!sed -i 's/max_batches = 500200/max_batches = 2000/g' cfg/yolov3.cfg
!sed -i 's/steps=400000,450000/steps=1600,1800/g' cfg/yolov3.cfg
!sed -i 's/classes=80/classes=1/g' cfg/yolov3.cfg
!sed -i 's/filters=255/filters=18/g' cfg/yolov3.cfg
!cat cfg/yolov3.cfg

sed: can't read cfg/yolov3.tiny.cfg: No such file or directory
sed: can't read cfg/yolov3.tiny.cfg: No such file or directory
[net]
# Testing
batch=64
subdivisions=1
# Training
# batch=64
# subdivisions=2
width=416
height=416
channels=3
momentum=0.9
decay=0.0005
angle=0
saturation = 1.5
exposure = 1.5
hue=.1

learning_rate=0.001
burn_in=1000
max_batches = 500200
policy=steps
steps=1600,1800
scales=.1,.1

[convolutional]
batch_normalize=1
filters=16
size=3
stride=1
pad=1
activation=leaky

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=32
size=3
stride=1
pad=1
activation=leaky

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=leaky

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=leaky

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=leaky

[maxpool]
size=2
stride=2

[convolutional]
batch_normaliz

Esta celda se utiliza para importar los pesos entrenados desde google drive a la carpeta darknet. (usar una vez ya entrenado el modelo)

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-07-14 17:04:21--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M   459KB/s    in 5m 11s  

2020-07-14 17:09:34 (510 KB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



Se comienza con el entrenamiento.

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3-tiny.cfg darknet53.conv.74 -dont_show 

Se han truncado las últimas 5000 líneas del flujo de salida.
 total_bbox = 123933, rewritten_bbox = 0.002421 % 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 16 Avg (IOU: 0.683573, GIOU: 0.654161), Class: 0.999867, Obj: 0.868633, No Obj: 0.000933, .5R: 1.000000, .75R: 0.500000, count: 2, class_loss = 0.028762, iou_loss = 0.347038, total_loss = 0.375800 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 23 Avg (IOU: 0.691108, GIOU: 0.663063), Class: 0.924999, Obj: 0.254365, No Obj: 0.000244, .5R: 1.000000, .75R: 0.000000, count: 1, class_loss = 0.443270, iou_loss = 0.268759, total_loss = 0.712028 
 total_bbox = 123936, rewritten_bbox = 0.002421 % 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 16 Avg (IOU: 0.725667, GIOU: 0.708653), Class: 0.998689, Obj: 0.554001, No Obj: 0.001118, .5R: 1.000000, .75R: 0.500000, count: 2, class_loss = 0.359065, iou_loss = 0.213339, total_loss = 0.572404 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 23 Avg (IOU: 0.0000

Se guardan los pesos copnseguidos en google drive para respaldar.

In [ ]:
import os.path
import shutil
from google.colab import drive

if not os.path.exists('/content/gdrive'):
  drive.mount('/content/gdrive')
  
YOLO_BACKUP = '/content/darknet/backup/yolov3_last.weights' 
DRIVE_DIR = '/content/gdrive/My Drive/ml2/' 

shutil.copy(YOLO_BACKUP, DRIVE_DIR)

print('Saved training data to drive at: ' + DRIVE_DIR)

celda de prueba. en esta etapa es necesario poseer una carpeta para testing y su respectivo txt (no se hizo en el dataset) se puede crear en la misma carpeta darknet en la ruta darknet/data crear carpeta y guardar algunas imagenes de prueba. se debe crear tambien un archivo test.txt en la misma ruta, que contenga la ruta de cada imagen guardada. (verificar nombre de la imagen) en caso de error con archivo obj.names mover dicho archvo a darknet/data 

recomendacion: mover los archivos obj.data obj.names img y train.txt del dataset a la ruta darknet/data para evitar errores (al hacer esto se deben cambiar las rutas de esta celda)

In [ ]:
!./darknet detector demo data/obj.data cfg/yolov3-tiny.cfg backup/yolov3-tiny_last.weights 1.mp4

 CUDA-version: 10010 (10010), GPU count: 1  
 OpenCV version: 3.2.0
Demo
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 1, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     16       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  16 0.150 BF
   1 max                2x 2/ 2    416 x 416 x  16 ->  208 x 208 x  16 0.003 BF
   2 conv     32       3 x 3/ 1    208 x 208 x  16 ->  208 x 208 x  32 0.399 BF
   3 max                2x 2/ 2    208 x 208 x  32 ->  104 x 104 x  32 0.001 BF
   4 conv     64       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  64 0.399 BF
   5 max                2x 2/ 2    104 x 104 x  64 ->   52 x  52 x  64 0.001 BF
   6 conv    128       3 x 3/ 1     52 x  52 x  64 ->   52 x  52 x 128 0.399 BF
   7 max                2x 2/ 2     52 x  52 x 128 ->   26 x  26 x 128 0.000 BF
   8 conv    256       3 x 3/ 1     26 x  26 x 128 ->   26 x  26 x 256 0.399 

La imagen testeada se guardará en prediction.jpg, en esta celda se visualiza. en caso de querer probar un video se utiliza la celda anterior con la ruta del video, y con OPENCV compilado en los pasos anteriores se visualizará inmediatamente. para usar con camara web, se deja la celda anterior sin ruta de prueba (aún no probado en google colab)

In [ ]:
def display_image(file_path = 'predictions.jpg'):
    import cv2
    import matplotlib.pyplot as plt
    import os.path

    if os.path.exists(file_path):
      img = cv2.imread(file_path)
      show_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
      plt.imshow(show_img)
    else:
      print('ERROR')
    
display_image()